Description: This code produces a two panel plot showing various variables from GFS analyses. Portions of this code are adapted from examples provided on the Unidata MetPy Python Gallery:
        https://unidata.github.io/python-gallery/examples/850hPa_Temperature_Advection.html

In [1]:
import cfgrib
import xarray
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
import scipy.ndimage as ndimage
#Create custom map to make middle (no change) white
colors = [(0, "blue"), (0.4, "white"), (0.6, "white"), (1, "red")]
bwr_white = LinearSegmentedColormap.from_list("bwwr", colors)

#Define data directory
BASE_DIR = 'data/GFS/'
#Open 12 UTC GFS analysis file
file = 'gfsanl_4_20190224_1200_000.grb2'
dt = xarray.open_dataset(BASE_DIR+file, engine='cfgrib', backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'meanSea'}))
mslp1200=dt['prmsl']/100. #Divide by 100 to get hPa / mb
lat=dt['latitude']
lon=dt['longitude']
#Open 18 UTC GFS analysis file
file = 'gfsanl_4_20190224_1800_000.grb2'
dt = xarray.open_dataset(BASE_DIR+file, engine='cfgrib', backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'meanSea'}))
mslp1800=dt['prmsl']/100.
#Load 850hPa data
dt = xarray.open_dataset(BASE_DIR+file, engine='cfgrib', backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'isobaricInhPa','level':925}))
t925 = dt['t'].values
u925 = dt['u'].values
v925 = dt['v'].values
#Gaussian filter to make contouring clean. 
t925 = ndimage.gaussian_filter(t925, sigma=1, order=0)
u925 = ndimage.gaussian_filter(u925, sigma=1, order=0)
v925 = ndimage.gaussian_filter(v925, sigma=1, order=0)

# Load sfc temperature data
dt = xarray.open_dataset(BASE_DIR+file, engine='cfgrib', backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'surface'}))
sfc_t=dt['t']-273.15
#Load 500 hPa data
dt = xarray.open_dataset(BASE_DIR+file, engine='cfgrib', backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'isobaricInhPa','level':500}))
hgt=dt['gh']
u=dt['u']
v=dt['v']
wind500=1.94384*np.sqrt(u*u+v*v) #Calculate magnitude of wind, convert to kts

#This code snippet is from the Metpy folks 
# Use helper function defined above to calculate distance
# between lat/lon grid points
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

# Because of the way the data are returned we need a negative spacing. This
# will be easier in the next version of MetPy.
dy *= -1

# Calculate temperature advection using metpy function
adv925 = mpcalc.advection(t925 , [u925, v925],(dx, dy), dim_order='yx')
sfc_t = ndimage.gaussian_filter(sfc_t, sigma=2, order=0) #smooth data for contouring purposes. 
mslpdif=mslp1800-mslp1200 #calculate 6hr change

#Contouring values
mdifval=[-20,-16,-12,-8,-4,-2,0,2,4,8,12,16,20]
advval=-20+4*np.arange(0,11,1)
mval=970+4*np.arange(0,20,1)
tval=[-20,-10,0,10,20,30]
wval=10*np.arange(0,14,1)
hval=5000.+100*np.arange(0,20,1)

#Create the plot
ef, (ax1,ax2) = plt.subplots(2,1,subplot_kw={'projection': ccrs.PlateCarree()},figsize=(6, 10),dpi=300)
ef.subplots_adjust(hspace=0,wspace=0,top=0.925,left=0.1)
ax1.set_extent([-117, -77, 30, 60], ccrs.PlateCarree())

cint = np.arange(-8, 9)
CS1 = ax1.contourf(lon,lat,adv925*21600., transform=ccrs.PlateCarree(), 
                   cmap=bwr_white,levels=advval) #Multiply by 21600 seconds = 6 hours 
#Had issue with contouring certain fields. Clipped domain to plotted area and problem went away...
CS2 = ax1.contour(lon[480:610],lat[55:125],hgt[55:125,480:610], transform=ccrs.PlateCarree(),colors="black",linewidths=2,levels=hval)
CS4 = ax1.contourf(lon,lat,wind500, transform=ccrs.PlateCarree(),cmap='Greys',alpha=0.3)
ax1.clabel(CS2, inline=1, fontsize=6,fmt='%4d')
ax1.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='gray')
ax1.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='black')

#Annotate domain for other plots
lonsb = [-98, -94, -94, -98,-98]
latsb = [50, 50, 43.5, 43.5,50]
ax1.plot(lonsb, latsb, transform=ccrs.PlateCarree(),linewidth=2,color="yellow",zorder=3)
#Note, code snippet below is thanks to stack overflow to get colorbar positioning right
#get size and extent of axes:
axpos = ax1.get_position()
pos_x = axpos.x0+axpos.width + 0.01# + 0.25*axpos.width
pos_y = axpos.y0
cax_width = 0.04
cax_height = axpos.height
#create new axes where the colorbar should go.
#it should be next to the original axes and have the same height!
pos_cax = ef.add_axes([pos_x,pos_y,cax_width,cax_height])
cbar1 = plt.colorbar(CS1, cax=pos_cax)
cbar1.ax.set_ylabel('925 hPa Temperature Advection ($^\circ$C/6hr)')
ax1.text(-116.8, 58.5, '(a)', fontsize=14,weight='bold',color="black")

#Plot 2
ef.subplots_adjust(hspace=0,wspace=0,top=0.925,left=0.1)
ax2.set_extent([-117, -77, 30, 60], ccrs.PlateCarree())

CS4 = ax2.contour(lon[480:610],lat[55:125],sfc_t[55:125,480:610], transform=ccrs.PlateCarree(),colors="red",
                  levels=tval,linewidths=1,linestyles='dashed',zorder=3)
CS5 = ax2.contour(lon,lat,mslp1800, transform=ccrs.PlateCarree(),colors="black",linewidths=2,levels=mval)
CS6 = ax2.contourf(lon,lat,mslpdif, transform=ccrs.PlateCarree(),cmap=bwr_white,alpha=0.8,levels=mdifval)
ax2.clabel(CS4, inline=1, fontsize=6,fmt='%4.0d')
ax2.clabel(CS5, inline=1, fontsize=6,fmt='%4.0d')
ax2.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='gray')
ax2.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='black')

ax2.plot(lonsb, latsb, transform=ccrs.PlateCarree(),linewidth=2,color="yellow",zorder=3)
#get size and extent of axes:
axpos = ax2.get_position()
pos_x = axpos.x0+axpos.width + 0.01# + 0.25*axpos.width
pos_y = axpos.y0
cax_width = 0.04
cax_height = axpos.height
#create new axes where the colorbar should go.
#it should be next to the original axes and have the same height!
pos_cax = ef.add_axes([pos_x,pos_y,cax_width,cax_height])
cbar2 = plt.colorbar(CS6, cax=pos_cax)
cbar2.ax.set_ylabel('6-hr MSLP Change (hPa)')
#cbar2.ax.set_ylabel('500 hPa Wind Magnitude (kts)')
ax2.text(-116.8, 58.5, '(b)', fontsize=14,weight='bold',color="black")

plt.savefig('KJ_MWR_2020_Fig01_final.png',dpi=300,bbox_inches='tight')
plt.savefig('KJ_MWR_2020_Fig01_final.pdf', format='pdf',bbox_inches='tight')
plt.show()


C:\ProgramData\Anaconda3\envs\SATPY\lib\site-packages\pint\quantity.py:802: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


<Figure size 1800x3000 with 4 Axes>